## AGL Wind Farm Power Prediction Notebook  

### This version is a modified version of https://github.com/osisoft/sample-ocs-data_views_jupyter-python/blob/main/Wind_Turbine_OCS_Data_OCS_Python_Library.ipynb for the [Academic Hub Wind Farms dataset](https://academic.osisoft.com/datasets)

#### Install the Academic Hub library and supporting modules

**NOTE: remove sharp sign (#) when notebook is run outside Binder**

In [ ]:
# !pip install ocs_academic_hub plotly matplotlib sklearn

### Import required modules and hub_login

In [ ]:
import requests
import json
import pandas as pd
from datetime import date, timedelta

import matplotlib.pyplot as plt
import numpy as np

from ocs_academic_hub.datahub import hub_login

### Login to Academic Hub by running the next cell

Simply follow the steps 

In [ ]:
widget, hub = hub_login()
widget

### Get latest dataset information

In [ ]:
hub.refresh_datasets()

### Standard Hub Datasets

In [ ]:
hub.datasets()

### Make WindFarm dataset the current dataset

In [ ]:
hub.set_dataset("MIT") 
## hub.set_dataset("<dataset>") # insert the correct name for the dataset to be used
hub.current_dataset()

### List the assets in the dataset

There are 10 wind turbines times 5 cluster (total of 50)

In [ ]:
hub.assets()

### Assets metadata

Store data about cluster no.1 into dataframe `df_meta` for map plot in next section

In [ ]:
df_metadata = hub.all_assets_metadata()
df_meta = df_metadata[df_metadata.Asset_Id.apply(lambda s: s[:8] == "cluster5")] # insert correct cluster
df_meta

#### Code for map generation using Plotly

With higher zoom to see turbine location and its ID better, including a different style to show terrain.

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(
    df_meta,
    lat="Latitude",
    lon="Longitude",
    text="Asset_Id",
    zoom=12.0,
    title="Locations of Cluster 1 wind turbines (red dots)",
)
fig.update_traces(marker=dict(size=12, color="green"))
# fig.update_layout(mapbox_style="stamen-terrain")
fig.update_layout(mapbox_style="open-street-map")
fig.show()

### Get the list of all single-asset data views

In [ ]:
hub.asset_dataviews()

### Get the list of all multiple-asset data views 

There is one per cluster 

In [ ]:
dataview_ids = hub.asset_dataviews("", multiple_asset=True)
dataview_id = dataview_ids[0]  # keep the first one for cluster1
dataview_ids

### Each dataset exist within a namespace

The next line finds and store it for the current dataset

In [ ]:
namespace_id = hub.namespace_of(hub.current_dataset())
namespace_id

### Verify the structure of the data view

For wind turbine `cluster1.turb1` (data view `wind.farms_cluster1` is similar but with all turbines of cluster no.1)

In [ ]:
hub.dataview_definition(namespace_id, "<dataview>") # insert dataview for cluster1.turb1

### Request data view result

For 2 months starting on 2019-01-01, interpolated every 5 minutes. Method `dataview_interpolated_pd` takes care of gathering multiple pages of data and returning a single Pandas dataframe.  

In [ ]:
dateFrom = "2019-01-01" # set start date
dateTo = "2019-03-01"  # set end date
timeinterval = "xx:xx:xx"  # interpolate every hour hh:mm:ss

df = hub.dataview_interpolated_pd(
    namespace_id, dataview_id, dateFrom, dateTo, timeinterval
)
df

In [ ]:
# Structure of dataframe
df.info()

In [ ]:
# Renaming DataFrame column names to abbreviations, in order to display these column names clearly in
# a correlation plot

df.rename(
    columns={
        "Rotor Speed": "RS",
        "State": "TS",
        "Power To Grid": "AP",  # Active Power
        "Pitch Angle": "NP",
        "Ambient Temperature": "AT",
        "Wind Speed": "WS",
    },
    inplace=True,
)
columns_to_drop = [
    c
    for c in df.columns
    if any(i in c for i in ["Drive", "Nacelle", "Yaw", "Relative"])
]
columns_to_drop
df.drop(columns_to_drop, axis=1, inplace=True)
df

In [ ]:
# Just keep the rows with the turbine in a good state

df = df[df["TS"] == "OK"]
df

In [ ]:
# Check the correlation between Active Power and the rest of the variables

# retrieve the correlation table
df_corr = df.corr()

# increase the size of the figure
fig = plt.figure(figsize=(50, 10))
ax = fig.add_subplot(111)

# set the color pallete (Red, yellow, green)
cax = ax.matshow(df_corr, cmap=plt.cm.RdYlGn)
fig.colorbar(cax)

# configure the labels
labels = [c for c in df_corr.columns]

# make sure to show all the labels
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))

# Setting labels for the x and y axes of the correlation plot
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

plt.show(block=False)

In [ ]:
# Renaming DataFrame column names from abbreviations back to their updated full names

df.rename(
    columns={
        "RS": "Rotor Speed",
        "TS": "State",
        "AP": "Active Power",
        "NP": "Nacelle Position",
        "AT": "Air Temperature",
        "WS": "Wind Speed",
    },
    inplace=True,
)
df

In [ ]:
# Plotting Best related variables

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.scatter(df["<insert variable 1>"], df["<insert variable 2>"])
ax.set_xlabel("<variable 1>")
ax.set_ylabel("<variable 2>")
ax.set_title("<variable 1> vs <variable 2>")
ax.set_xlim([0,20])

plt.show(block=False)

In [ ]:
# Remove any rows with missing data (if any)
df_Filter = df.dropna() 

In [ ]:
# Filter out negative & excessive Active Power Values
filterNegativeActivePower = df_Filter["Active Power"] >= 0
df_Filter = df_Filter[filterNegativeActivePower]
df_Filter

In [ ]:
# Remove the rows where we have a high wind speed and low active power in order to keep only the normal operating conditions
filterOutLowPowerHighWindSpeedData = ~(
    (df_Filter["Wind Speed"] > 10) & (df_Filter["Active Power"] < 600)
)
df_Filter = df_Filter[filterOutLowPowerHighWindSpeedData]

In [ ]:
df_Filter

In [ ]:
# Filter out high Wind Speeds (> 13 m/s) that do not change the Active Power results
filterOutHighWind = df_Filter["Wind Speed"] < 13
df_Filter = df_Filter[filterOutHighWind]
df_Filter

In [ ]:
# Plotting Active Power versus Wind Speed - filtered data frame representing Normal Operating Conditions

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.scatter(df_Filter["Wind Speed"], df_Filter["Active Power"])
ax.set_xlabel("Wind Speed (m/s)")
ax.set_ylabel("Active Power (kW)")
ax.set_title("Active Power vs Wind Speed")

plt.show(block=False)

In [ ]:
# Prepare the training & testing/scoring data sets, and split them randomly
from sklearn.model_selection import train_test_split

# define the target variable to be predicted
y = df_Filter["Active Power"].values
# split the dataset randomly into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    df_Filter[["Air Temperature", "Wind Speed"]].values,
    y,
    test_size=0.25,
    random_state=42,
)
print("-- training/testing prepared --")

In [ ]:
# Use the Decision Tree Regression Machine Learning model from scikit-learn
from sklearn.tree import DecisionTreeRegressor

regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)

# Predict
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)
print("-- decision tree regression --")

In [ ]:
# Plot the results
plt.figure(figsize=(10, 10))
plt.scatter(
    X_train[:, 1], y_train, s=20, edgecolor="black", c="darkorange", label="data"
)
plt.plot(X_test[:, 1], y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test[:, 1], y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("Wind Speed")
plt.ylabel("Active Power")
plt.title("Decision Tree Regression")
plt.legend()
plt.show(block=False)

In [ ]:
# save the machine learning model to disk
import pickle

filename = "WT_ActivePower_model.sav"
pickle.dump(regr_2, open(filename, "wb"))
print("-- model saved --")

In [ ]:
# Test the model with the scoring/testing data set
loaded_model = pickle.load(open(filename, "rb"))
global result
result = loaded_model.score(X_test, y_test)
# print the model score
print(result)

In [ ]:
# Sample prediction
# define input
new_input = [[<temp>, <wind speed>]]  # insert correct values for temp (°C) and wind speed (m/s) without <>
# get prediction for new input
new_output = regr_2.predict(new_input)
print(new_output)

In [ ]:
# Call the OpenWeather API to retrieve the forecasted air temperature and wind speed
# for Jamestown, Australia for the next 5 days
# City code information: http://bulk.openweathermap.org/sample/
#
url = "https://api.openweathermap.org/data/2.5/forecast?q=Jamestown,AU,2069194&units=metric&APPID=5dac981ce33f41f61d8d1ea06ee89798"
responseWeatherForecast = requests.get(url)

In [ ]:
# Display first 3 results 
responseWeatherForecast.json()["list"][:3]

In [ ]:
# Store the forecasted air temperature, wind speed and timestamp from the API json response in a pandas DataFrame

import datetime

TempArray = []
WindSpeedArray = []
TimestampArray = []

for val in responseWeatherForecast.json()["list"]:
    tempC = val["main"]["temp"]
    windSpeedMeterPerSec = round(val["wind"]["speed"], 2)
    np.array(TempArray.append(tempC))
    np.array(WindSpeedArray.append(windSpeedMeterPerSec))
    np.array(
        TimestampArray.append(
            datetime.datetime.strptime(val["dt_txt"], "%Y-%m-%d %H:%M:%S")
        )
    )

dfWeatherForecast = pd.DataFrame(
    {
        "Timestamp": TimestampArray,
        "Temp (C)": TempArray,
        "Wind Speed (m/s)": WindSpeedArray,
    }
)

dfWeatherForecast

In [ ]:
# Use the machine learning model developed previously to predict the Active Power
# and add the values to the existing Data Frame

import pickle

filename = "WT_ActivePower_model.sav"
loaded_model = pickle.load(open(filename, "rb"))

PredictedPowerArray = []

for index, row in dfWeatherForecast.iterrows():
    new_input = [[row["Temp (C)"], row["Wind Speed (m/s)"]]]
    result = loaded_model.predict(new_input)
    np.array(PredictedPowerArray.append(result))

dfWeatherForecast["Predicted Active Power (kW)"] = pd.DataFrame(PredictedPowerArray)

dfWeatherForecast

In [ ]:
# Plot predictions

import plotly.express as px

px.scatter_3d(
    dfWeatherForecast,
    x="Temp (C)",
    y="Wind Speed (m/s)",
    z="Predicted Active Power (kW)",
    size="Predicted Active Power (kW)",
    color="Predicted Active Power (kW)",
    log_x=False,
    size_max=100,
    range_x=[0, 90],
    range_y=[0, 12],
    height=800,
)